### Generating Synthetic Data using Sudoku

#### Create a new sudoku game

In [8]:
from utils.sudoku import SudokuEnvironment

game = SudokuEnvironment()
game.create_puzzle(difficulty='easy')
_ = game.display()

+-------+-------+
| _ 2 | _ 4 |
| _ 4 | 1 _ |
+-------+-------+
| 2 _ | 4 3 |
| 4 3 | 2 _ |
+-------+-------+


#### Build Data generation pipeline

##### Build question

In [2]:
import random
import copy
from random import randint

def randomize_dict_values(dict_list, seed=None):
    """
    Replace each 'value' with a random different value from range 1-4.
    The new value is guaranteed to be different from the original.
    All other keys remain unchanged.

    Args:
        dict_list: List of dictionaries containing 'value' key
        seed: Random seed for reproducibility (optional)

    Returns:
        List of dictionaries with 'value' key randomized to different values
    """
    if not dict_list:
        return []

    if seed is not None:
        random.seed(seed)

    # Create deep copy to avoid modifying the original
    randomized_list = copy.deepcopy(dict_list)

    # Replace each value with a random different value
    for new_dict in randomized_list:
        original_value = new_dict['value']

        # Create possible values (1-4) excluding the original value
        possible_values = [v for v in range(1, 5) if v != original_value]

        # Randomly select from the remaining values
        new_dict['value'] = random.choice(possible_values)

    return randomized_list

def generate_answer_options(correct_deduction, wrong_deductions):
    """
    Generate multiple-choice answer options with one correct and several wrong answers.

    Args:
        correct_deduction: Dictionary containing the correct deduction
        wrong_deductions: List of dictionaries containing wrong deductions

    Returns:
        tuple: (formatted_answer_string, correct_answer_letter)
    """
    answer_options_list = []
    answer_options_set = {'A', 'B', 'C', 'D'}

    # Select a random option for the correct answer
    correct_answer_option = random.choice(list(answer_options_set))
    answer_options_set.discard(correct_answer_option)

    # Add correct answer to the list
    correct_text = f"{correct_answer_option}) Position ({correct_deduction['row']},{correct_deduction['col']}) must be {correct_deduction['value']}"
    answer_options_list.append(correct_text)

    # print(len(wrong_deductions))
    for idx, wrong_deduction in enumerate(wrong_deductions):
        wrong_answer_option = random.choice(list(answer_options_set))
        answer_options_set.discard(wrong_answer_option)
        # print(wrong_deduction)
        wrong_text = f"{wrong_answer_option}) Position ({wrong_deduction['row']},{wrong_deduction['col']}) must be {wrong_deduction['value']}"
        answer_options_list.append(wrong_text)
        if idx == 2:
            break

    # Sort by the option letter and join into a string
    answer_options_list.sort(key=lambda x: x[0])
    formatted_answers = "\n".join(answer_options_list)

    # Extract the correct answer letter (without the closing parenthesis)
    correct_letter = correct_answer_option[0]

    return formatted_answers, correct_letter

In [3]:
# 1. First we define the schema for the output data

puzzle = {
    "question": None,
    "question_parsing": None,
    "answer": None,
    "id": None,
    "sel_idx": None,
    "cot": None,
    "cot_parsing": None
}

# 2. Let's create the questions with
# define overall question
question = 'You have the following Sudoku puzzle which of the following answers is correct? \n'

# Append the game to the question
question += game.display(print_game=False)

deductions_data = game.get_explicit_deductions()[-1]
correct_deduction = deductions_data[0]
wrong_deductions = randomize_dict_values(deductions_data[1:])
opts, answer = generate_answer_options(correct_deduction, wrong_deductions)
question += "\n" + opts
print(question)

puzzle['question'] = question
puzzle['answer'] = answer

# Get permutations -> create a questions for each of the empty fields -> for possible answers a, b, c, d create one correct answer and 3 wrong answers


# 3. Let's create the question parsing
qp_raw = game.get_all_units_as_strings()
qp_string = 'The puzzle has the rows: ' + str(qp_raw['rows'])
qp_string += '\nThe puzzle has the columns: ' + str(qp_raw['columns'])
qp_string += '\nThe puzzle has the squares: ' + str(qp_raw['squares'])

print(qp_string)

puzzle['question_parsing'] = qp_string

# 4. Generating cot
# Get full deduction
cot = game.get_explicit_deductions()[0]
cot_string = "; ".join(cot)
puzzle['cot'] = cot_string

# 5. Generating cot parsing
# Get statements from deduction
statements = game.get_explicit_deductions()[1]
# get evidence from deduction
evidence = game.get_explicit_deductions()[2]

# Add verification -> wrong verifications?
puzzle['cot_parsing'] = [{"statement": statements[idx], "evidence": evidence[idx],"Verification": "true"} for idx in range(len(statements))]


# 6. Generate ids

# add random ids
rand =  randint(0, 100000000)
puzzle['id'] = rand
puzzle['sel_idx'] = rand
print("--------")

puzzle

You have the following Sudoku puzzle which of the following answers is correct? 
+-------+-------+
| 1 _ | 3 4 |
| 3 4 | 1 _ |
+-------+-------+
| _ _ | 4 _ |
| 4 _ | 2 1 |
+-------+-------+
A) Position (2,3) must be 2
B) Position (2,0) must be 1
C) Position (1,3) must be 4
D) Position (0,1) must be 2
The puzzle has the rows: ['1 _ 3 4', '3 4 1 _', '_ _ 4 _', '4 _ 2 1']
The puzzle has the columns: ['1 3 _ 4', '_ 4 _ _', '3 1 4 2', '4 _ _ 1']
The puzzle has the squares: [' 1_34', ' 341_', ' __4_', ' 4_21']
--------


{'question': 'You have the following Sudoku puzzle which of the following answers is correct? \n+-------+-------+\n| 1 _ | 3 4 |\n| 3 4 | 1 _ |\n+-------+-------+\n| _ _ | 4 _ |\n| 4 _ | 2 1 |\n+-------+-------+\nA) Position (2,3) must be 2\nB) Position (2,0) must be 1\nC) Position (1,3) must be 4\nD) Position (0,1) must be 2',
 'question_parsing': "The puzzle has the rows: ['1 _ 3 4', '3 4 1 _', '_ _ 4 _', '4 _ 2 1']\nThe puzzle has the columns: ['1 3 _ 4', '_ 4 _ _', '3 1 4 2', '4 _ _ 1']\nThe puzzle has the squares: [' 1_34', ' 341_', ' __4_', ' 4_21']",
 'answer': 'D',
 'id': 65139684,
 'sel_idx': 65139684,
 'cot': "Position (0,1) must be 2 (only valid value  because 10 - 1 - 3 - 4 = 2); Position (1,3) must be 2 (only valid value  because 10 - 1 - 3 - 4 = 2); Position (2,0) must be 2 (only valid value  because 10 - 1 - 3 - 4 = 2); Position (2,3) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Position (3,1) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Positio

In [11]:
import random
from random import randint

def generate_sudoku_puzzles(num_puzzles, difficulty='easy', seed=None, verbose=False):
    """
    Generate multiple Sudoku puzzles with questions, answers, and explanations.

    Args:
        num_puzzles (int): Number of puzzles to generate
        difficulty (str): Difficulty level for puzzle generation ('easy', 'medium', 'hard')
        seed (int, optional): Random seed for reproducibility
        verbose (bool): Whether to print progress and details

    Returns:
        list: List of puzzle dictionaries with question, answer, parsing, etc.
    """

    if seed is not None:
        random.seed(seed)

    puzzles = []
    successful_puzzles = 0
    attempts = 0
    max_attempts = num_puzzles * 10  # To prevent infinite loops
    grid_signatures = []

    if verbose:
        print(f"Generating {num_puzzles} puzzles with difficulty '{difficulty}'...")

    while successful_puzzles < num_puzzles and attempts < max_attempts:
        attempts += 1

        try:
            # Create new game instance
            game = SudokuEnvironment()

            # Generate puzzle
            if not game.create_puzzle(difficulty=difficulty):
                if verbose and attempts % 10 == 0:
                    print(f"Attempt {attempts}: Failed to create puzzle, retrying...")
                continue

            grid_signature = game.get_grid_signature()

            if grid_signature not in grid_signatures:
                grid_signatures.append(grid_signature)

                # Get deductions data
                deductions_data = game.get_explicit_deductions()

                # Check if we have enough deductions
                if len(deductions_data) < 2:
                    if verbose:
                        print(f"Attempt {attempts}: Not enough deductions, skipping...")
                    continue

                # Initialize puzzle dictionary
                puzzle = {
                    "question": None,
                    "question_parsing": None,
                    "answer": None,
                    "id": None,
                    "sel_idx": None,
                    "cot": None,
                    "cot_parsing": None
                }

                # 1. Create the question
                question = 'You have the following Sudoku puzzle which of the following answers is correct? \n'
                question += game.display()  # Assuming display() returns string now

                # Get correct and wrong deductions
                all_deductions = deductions_data[-1]  # Get the structured deductions
                correct_deduction = all_deductions[0]

                if len(all_deductions) > 1:
                    wrong_deductions = randomize_dict_values(all_deductions[1:])
                else:
                    # If only one deduction -> create variations
                    wrong_deductions = randomize_dict_values([correct_deduction] * 3)

                # Generate answer options
                opts, answer = generate_answer_options(correct_deduction, wrong_deductions)
                question += "\n" + opts

                puzzle['question'] = question
                puzzle['answer'] = answer

                # 2. Create question parsing
                qp_raw = game.get_all_units_as_strings()
                qp_string = 'The puzzle has the rows: ' + str(qp_raw['rows'])
                qp_string += '\nThe puzzle has the columns: ' + str(qp_raw['columns'])
                qp_string += '\nThe puzzle has the squares: ' + str(qp_raw['squares'])

                puzzle['question_parsing'] = qp_string

                # 3. Generate chain of thought (cot)
                cot = deductions_data[0]  # Get full deduction strings
                cot_string = "; ".join(cot)
                puzzle['cot'] = cot_string

                # 4. Generate cot parsing
                statements = deductions_data[1]  # Get statements
                evidence = deductions_data[2]    # Get evidence

                puzzle['cot_parsing'] = [
                    {
                        "statement": statements[idx],
                        "evidence": evidence[idx],
                        "Verification": "true" # todo for later to also include wrong statements
                    }
                    for idx in range(len(statements))
                ]

                # 5. Generate unique IDs
                rand_id = randint(0, 100000000)
                puzzle['id'] = rand_id
                puzzle['sel_idx'] = rand_id

                # Add puzzle to list
                puzzles.append(puzzle)
                successful_puzzles += 1

                if verbose:
                    print(f"Successfully generated puzzle {successful_puzzles}/{num_puzzles}")
            else:
                print("Puzzle already generated")

        except Exception as e:
            if verbose:
                print(f"Attempt {attempts}: Error generating puzzle - {str(e)}")
            continue

    if successful_puzzles < num_puzzles:
        print(f"Warning: Only generated {successful_puzzles} out of {num_puzzles} requested puzzles after {attempts} attempts")

    return puzzles


In [12]:
puzzles = generate_sudoku_puzzles(
    num_puzzles=250,
    difficulty='easy',
    seed=42,
    verbose=True
)

print(f"\nGenerated {len(puzzles)} puzzles")


# Example of how a puzzle could look like
puzzles[0]

Generating 250 puzzles with difficulty 'easy'...
+-------+-------+
| _ 2 | 3 _ |
| _ 4 | 1 2 |
+-------+-------+
| 2 1 | 4 _ |
| 4 _ | 2 _ |
+-------+-------+
Successfully generated puzzle 1/250
+-------+-------+
| _ 2 | _ _ |
| 3 4 | 1 2 |
+-------+-------+
| _ _ | 4 3 |
| 4 3 | _ 1 |
+-------+-------+
Successfully generated puzzle 2/250
+-------+-------+
| 1 2 | _ 4 |
| _ _ | _ 2 |
+-------+-------+
| 2 1 | 4 _ |
| 4 3 | 2 _ |
+-------+-------+
Successfully generated puzzle 3/250
+-------+-------+
| 1 _ | 3 4 |
| 3 4 | _ 2 |
+-------+-------+
| _ 1 | 4 3 |
| 4 _ | _ _ |
+-------+-------+
Successfully generated puzzle 4/250
+-------+-------+
| 1 _ | 3 _ |
| _ 4 | 1 _ |
+-------+-------+
| 2 1 | 4 3 |
| _ 3 | _ 1 |
+-------+-------+
Successfully generated puzzle 5/250
+-------+-------+
| 1 _ | 3 4 |
| 3 4 | 1 2 |
+-------+-------+
| _ _ | _ _ |
| 4 _ | 2 1 |
+-------+-------+
Successfully generated puzzle 6/250
+-------+-------+
| _ 2 | 3 _ |
| 3 _ | 1 _ |
+-------+-------+
| 2 1 | _ 3

{'question': 'You have the following Sudoku puzzle which of the following answers is correct? \n+-------+-------+\n| _ 2 | 3 _ |\n| _ 4 | 1 2 |\n+-------+-------+\n| 2 1 | 4 _ |\n| 4 _ | 2 _ |\n+-------+-------+\nA) Position (0,0) must be 1\nB) Position (1,0) must be 4\nC) Position (0,3) must be 1\nD) Position (2,3) must be 1',
 'question_parsing': "The puzzle has the rows: ['_ 2 3 _', '_ 4 1 2', '2 1 4 _', '4 _ 2 _']\nThe puzzle has the columns: ['_ _ 2 4', '2 4 1 _', '3 1 4 2', '_ 2 _ _']\nThe puzzle has the squares: [' _2_4', ' 3_12', ' 214_', ' 4_2_']",
 'answer': 'A',
 'id': 3999315,
 'sel_idx': 3999315,
 'cot': "Position (0,0) must be 1 (only valid value  because 10 - 2 - 3 - 4 = 1); Position (0,3) must be 4 (only valid value  because 10 - 1 - 2 - 3 = 4); Position (1,0) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Position (2,3) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Position (3,1) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Position 